In [18]:
import numpy as np
n_features = 'all'
term_pred = 'short_term'
data_file_name = 'data_y10'
endpt = 10
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'

In [19]:
# %pip install tensorflow
# %pip install lifelines

# !y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [20]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [21]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [22]:
load_dir = work_dir+ '/csv_files'
# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [23]:
data_full

,ID,event,time,sex,race,cholesterol,hdl,hbp.medication,smoker,diabetes,...,C08MELAG_YEAR,C08SKNAG_YEAR,C08BRNAG_YEAR,C08STMAG_YEAR,C08OCAAG_YEAR,C08GALDA_YEAR,C08PRGAG_YEAR,C08KYSAG_YEAR,C08URNAG_YEAR,count_na
0,100016012504,0,3689.025,0,0,186.0,38.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.048673
1,100023004268,0,3689.025,0,0,166.0,43.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055310
2,100033323702,0,3689.025,0,0,234.0,52.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.077434
3,100048415270,0,1169.000,0,1,137.0,40.0,0,1,0,...,0,0,0,0,0,0,0,0,18,0.008850
4,100056526386,0,3689.025,0,1,199.0,44.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.008850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,1,0,225.0,57.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.048673
4310,416771521620,0,3689.025,0,1,204.0,76.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.004425
4311,416783315386,0,3689.025,1,0,159.0,58.0,0,0,0,...,0,0,0,0,0,0,8,0,0,0.112832
4312,416796224310,0,3689.025,1,1,164.0,55.0,0,0,0,...,0,0,0,0,0,0,14,0,0,0.070796


In [24]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/all_training_ID_outerloop_cohort_0_10.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [25]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,444,445,446,447,448,449,450,451,452,453
0,100016012504,0,3689.025,-1.106215,-0.970306,0.235193,-1.090061,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.773715
1,100023004268,0,3689.025,-1.106215,-0.970306,-0.356134,-0.732871,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,0.979473
2,100033323702,0,3689.025,-1.106215,-0.970306,1.654379,-0.089929,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,1.665334
3,100048415270,0,1169.000,-1.106215,1.030602,-1.213559,-0.947185,-0.126551,1.580754,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,8.591018,-0.460833
4,100056526386,0,3689.025,-1.106215,1.030602,0.619556,-0.661433,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.460833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,0.903984,-0.970306,1.388282,0.267261,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,0.773715
4310,416771521620,0,3689.025,-1.106215,1.030602,0.767388,1.624583,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,-0.626333,-0.090552,-0.124130,-0.598005
4311,416783315386,0,3689.025,0.903984,-0.970306,-0.563099,0.338699,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,0.856246,-0.090552,-0.124130,2.762710
4312,416796224310,0,3689.025,0.903984,1.030602,-0.415267,0.124385,-0.126551,-0.632609,-0.150863,...,-0.022243,-0.024272,-0.021537,-0.015227,-0.056678,-0.087086,1.968180,-0.090552,-0.124130,1.459575


In [26]:
# decide on number of dicrete times:
#halflife=13.*365.25

#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [27]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features


def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [13]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    l2_final=0.0001

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred+'/l1_1e-4')
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)





FOLD 0:

Train C-index fold 1 :
0.9676516446565101
Test C-index fold 1 :
0.5114754098360655

FOLD 1:

Train C-index fold 2 :
0.9758327711634183
Test C-index fold 2 :
0.7670160631636265

FOLD 2:

Train C-index fold 3 :
0.9137357933848171
Test C-index fold 3 :
0.6377732240437158

FOLD 3:

Train C-index fold 4 :
0.951811399473478
Test C-index fold 4 :
0.4788741992640044

FOLD 4:

Train C-index fold 5 :
0.9639948681085705
Test C-index fold 5 :
0.7268429706768977

FOLD 5:

Train C-index fold 6 :
0.9560873562355766
Test C-index fold 6 :
0.5750518229484209

FOLD 6:

Train C-index fold 7 :
0.9196984431377182
Test C-index fold 7 :
0.8974800330487469

FOLD 7:

Train C-index fold 8 :
0.9436735696607949
Test C-index fold 8 :
0.6594928032899247

FOLD 8:

Train C-index fold 9 :
0.9652565773286067
Test C-index fold 9 :
0.5797654758658304

FOLD 9:

Train C-index fold 10 :
0.9572370220447459
Test C-index fold 10 :
0.5496261046906866

FOLD 10:

Train C-index fold 11 :
0.9357174322589111
Test C-index fo